Import all necessary libraries and modules.

In [7]:
import sys
!{sys.executable} -m pip install --upgrade nbformat>=5.0.0
sys.path.append('../')

import numpy as np
import seaborn as sb
import matplotlib as mb
import matplotlib.pyplot as plt
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import feature_engine as fe
import sys
from assets.transformers import pipeline, scaling_transformer

Call the recently created dataset to work from.

In [8]:
bc = pd.read_csv("../assets/processed-bank-churners.csv")
bc

,Customer_Status,Age,Gender,Income_Level,Tenure_Months,Inactive_Months_In_Last_12,Credit_Limit,Total_Trans_Amount,Total_Trans_Count,Avg_Utilization_Ratio
0,Existing Customer,41,F,Unknown,30,1,5417.0,1768,56,0.35
1,Attrited Customer,60,M,$40K - $60K,47,1,3735.0,8193,66,0.32
2,Existing Customer,47,M,Less than $40K,34,1,8390.0,8316,92,0.00
3,Existing Customer,44,M,$60K - $80K,37,2,1682.0,3309,67,0.55
4,Existing Customer,47,F,Less than $40K,36,2,2500.0,4265,87,0.47
...,...,...,...,...,...,...,...,...,...,...
191,Existing Customer,31,M,$40K - $60K,22,1,9096.0,2318,54,0.00
192,Existing Customer,50,F,Less than $40K,38,3,2775.0,4608,84,0.53
193,Existing Customer,38,F,Less than $40K,29,3,3590.0,1472,37,0.54
194,Attrited Customer,39,F,$40K - $60K,31,3,3333.0,3348,50,0.56


Create interactive visualizations using Plotly.

In [58]:
bc_numeric = bc.copy()
bc_numeric['Customer_Status_Numeric'] = bc['Customer_Status'].map({
    'Existing Customer': 0, 
    'Attrited Customer': 1
})
fig = px.parallel_coordinates(bc_numeric, 
                            dimensions=['Avg_Utilization_Ratio', bc.columns[-1], 'Credit_Limit'],
                            color='Customer_Status_Numeric',
                            title='Parallel Coordinates: Usage, Utilization & Credit Limit by Customer Status',
                            color_continuous_scale=[[0, 'blue'], [1, 'red']])
fig.update_layout(
    width=800,
    height=600
)
fig.show()

As we can see from the parallel coordinates graph above, 

In [16]:
# Create custom colors with red for attrited customers
custom_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#F38BA8', '#A8E6CF']

fig = px.sunburst(bc, 
                 path=['Customer_Status', 'Gender', 'Income_Level'],
                 title='Comparison Between Customer Status, Gender and Income Level',
                 color_discrete_sequence=custom_colors)

fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  '<extra></extra>',
    textinfo="label+percent parent",
    marker=dict(
        line=dict(
            color="darkred",
            width=2
        )
    )
)

fig.update_layout(
    width=700,
    height=700
)

fig.show()

This graph ghjkjhgfghjkjhg

In [11]:
# Interactive scatter plot: Customer Status Analysis
fig = px.scatter(bc, 
                x='Total_Trans_Count', 
                y='Total_Trans_Amount',
                color='Customer_Status',
                title='Transaction Count vs Transaction Amount by Customer Status',
                labels={
                    'Total_Trans_Count': 'Transaction Count',
                    'Total_Trans_Amount': 'Total Transaction Amount ($)'
                },
                color_discrete_map={
                    'Existing Customer': 'blue',
                    'Attrited Customer': 'red'
                })

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGray')))
fig.update_layout(
    width=900,
    height=600,
    xaxis_title='Transaction Count',
    yaxis_title='Total Transaction Amount ($)'
)

fig.show()

The scatter plot allows for the visualisation of 

In [63]:
bc_numeric = bc.copy()
bc_numeric['Customer_Status_Numeric'] = bc['Customer_Status'].map({
    'Existing Customer': 0, 
    'Attrited Customer': 1
})

fig = px.parallel_coordinates(bc_numeric, 
                            dimensions=['Avg_Utilization_Ratio', bc.columns[-1], 'Credit_Limit'],
                            color='Customer_Status_Numeric',
                            title='Parallel Coordinates: Usage, Utilization & Credit Limit by Customer Status',
                            color_continuous_scale=[[0, 'blue'], [1, 'red']])

# Remove the default colorbar
fig.update_coloraxes(showscale=False)

# Add custom legend using dummy traces
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                        line=dict(color='blue', width=3),
                        name='Existing Customer',
                        showlegend=True))

fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                        line=dict(color='red', width=3),
                        name='Attrited Customer',
                        showlegend=True))

fig.update_layout(
    width=900,
    height=600,
    legend=dict(
        x=1.02,
        y=1,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='black',
        borderwidth=1
    ),
    margin=dict(r=150)
)

fig.show()

cfghjkjhgf